#### Product recommendations in natural language using RAG and LLM
 - Embed user query using same embedding model as the vector database.(Sentence-Transformers)
 - Retrieve similar products using similarity search by query or by embeddings. (ChromaDB)
 - Generate a Natural Language Response using an LLM.
    - Open source models: LLaMA-2/Mistral/Gemma/Phi-2
    - ChatOpen AI  gpt-4

#### Load Pretrained Embedding Model

In [1]:
import pandas as pd
import os

In [2]:
# Define the storage path
PERSIST_DIRECTORY = "chromadb_vectorstore"

In [4]:
# Run this if you want to delete existing Chroma DB and start fresh DB creation. Restart before creating DB
import shutil

# Delete the existing directory
if os.path.exists(PERSIST_DIRECTORY):
    shutil.rmtree(PERSIST_DIRECTORY)
    print(f"Deleted existing ChromaDB at: {PERSIST_DIRECTORY}")

# If deleted, manually create the PERSIST_DIRECTORY again and restart to run recreation of the ChromaDB.

Deleted existing ChromaDB at: chromadb_vectorstore


In [ ]:
SAMPLE_FILE = "sample_20k.csv"

df_sample = pd.read_csv(SAMPLE_FILE)
df_sample.head(5)

,item_id,item_name,product_type,country,enhanced_product_desc,image_path,image_caption,complete_product_description
0,B07TG4V6BV,Amazon Brand - Solimo Designer Black and White...,CELLULAR_PHONE_CASE,IN,"Given Product description: , No Warranty, bran...",5f/5f39a379.jpg,a black and white phone case with a design on it,a black and white phone case with a design on ...
1,B07T2K5MY1,Amazon Brand - Solimo Designer Galaxy 3D Print...,CELLULAR_PHONE_CASE,IN,"Given Product description: , No Warranty, bran...",4a/4ab3ead6.jpg,a phone case with an orange swirl design,a phone case with an orange swirl design Given...
2,B0854LLTNR,Amazon Brand - Solimo Designer Daddy's Girl an...,CELLULAR_PHONE_CASE,IN,"Given Product description: , Extreme precision...",cd/cd678bbf.jpg,a black and white phone case with the words da...,a black and white phone case with the words da...
3,B07TGBPM1H,Amazon Brand - Solimo Designer Kiss-ing Couple...,CELLULAR_PHONE_CASE,IN,"Given Product description: , None, brand: Amaz...",d4/d47d521f.jpg,a couple is silhouetted against each other pho...,a couple is silhouetted against each other pho...
4,B077VJDKLV,"find. Women’s Flat Mule Sandals, Black, 8 UK",SANDAL,GB,"Given Product description: , Leather look uppe...",8e/8e2b2da0.jpg,a pair of black sandals on a white background,a pair of black sandals on a white background ...


In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


PERSIST_DIRECTORY = "chromadb_vectorstore"
# Initialize ChromaDB and OpenAI embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma(embedding_function=embeddings, persist_directory=PERSIST_DIRECTORY)  # Pass embeddings to Chroma

# Store product embeddings
docs = [{'id': i, 'text': row['complete_product_description']} for i, row in df_sample.iterrows()]
vectorstore.add_texts(texts=[doc['text'] for doc in docs], metadatas=docs)

print("Embeddings generated and vector store saved to:", PERSIST_DIRECTORY)

/var/folders/y4/z3s1vk012bv7p_188zpx6w3c0000gn/T/ipykernel_3420/1853078797.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/Users/mitilroy/Dev/src/capstone/master/ShopTalk/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/y4/z3s1vk012bv7p_188zpx6w3c0000gn/T/ipykernel_3420/1853078797.py:8: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 

Embeddings generated and vector store saved to: chromadb_vectorstore


In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Initialize ChromaDB and OpenAI embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma(embedding_function=embeddings, persist_directory=PERSIST_DIRECTORY)  # Pass embeddings to Chroma

### Retrieve similar products

In [8]:
### Retrieve Similar Products
def retrieve_similar_products(query, top_k=5):
    """Retrieve the top-K similar products based on the query."""
    results = vectorstore.similarity_search(query, k=top_k)  # Get top 5 matches
    
    recommended_products = []
    for doc in results:
        id = doc.metadata.get("id")
        image_path = df_sample.iloc[id]['image_path']
        if not image_path or pd.isna(image_path):
            image_path = "No image available"
        recommended_products.append({
            "product_description": doc.page_content,  # Retrieved text
            "product_image": image_path,  # Image link
            "product_type": df_sample.iloc[id]['product_type'],
            "country": df_sample.iloc[id]['country'],
        })
        print(f"Doc: {doc}")
        print(f"metadata: {doc.metadata}")
        print(f"image_url: {image_path}")
        print(f"product_type: {df_sample.iloc[id]['product_type']}")
        print(f"country: {df_sample.iloc[id]['country']}")
        print("--------")
    
    return recommended_products

# Example user query
query = "Find me Men's Sneaker in black color"
recommended_products = retrieve_similar_products(query)

Doc: page_content='men ' s sneakers in grey and black Given Product description: , Care Instructions: Allow your pair of shoes to air and de-odorize at regular basis; Using a Shoe-horn to wear your shoes will avoid damage to the back of your shoes; Use Shoe bags to prevent any stains or mildew., brand: Amazon Brand - Inkast Denim Co., weight: , color: L.GREY, height: 1286.0, width: 2342.0, model year: , shape: , style: Sneakers, material: , product_type: SHOES' metadata={'id': 1435, 'text': "men ' s sneakers in grey and black Given Product description: , Care Instructions: Allow your pair of shoes to air and de-odorize at regular basis; Using a Shoe-horn to wear your shoes will avoid damage to the back of your shoes; Use Shoe bags to prevent any stains or mildew., brand: Amazon Brand - Inkast Denim Co., weight: , color: L.GREY, height: 1286.0, width: 2342.0, model year: , shape: , style: Sneakers, material: , product_type: SHOES"}
metadata: {'id': 1435, 'text': "men ' s sneakers in gre

In [9]:
df_sample.iloc[1597]

item_id                                                                B08547752Y
item_name                       Amazon Brand - Solimo Designer Water Drops UV ...
product_type                                                  CELLULAR_PHONE_CASE
country                                                                        IN
enhanced_product_desc           Given Product description: , Extreme precision...
image_path                                                        3f/3f9ae223.jpg
image_caption                   the back of a phone case with water droplets o...
complete_product_description    the back of a phone case with water droplets o...
Name: 1597, dtype: object

In [10]:
df_sample.iloc[1597]['image_path']

'3f/3f9ae223.jpg'

In [11]:
# Print the recommended products
for i, result in enumerate(recommended_products):
    print(f"Result {i+1}:")
    print(f"Product Description: {result['product_description']}")
    print(f"Product Image: {result['product_image']}")
    print(f"Category: {result['product_type']}")
    print(f"Country: {result['country']}")
    print("---")

Result 1:
Product Description: men ' s sneakers in grey and black Given Product description: , Care Instructions: Allow your pair of shoes to air and de-odorize at regular basis; Using a Shoe-horn to wear your shoes will avoid damage to the back of your shoes; Use Shoe bags to prevent any stains or mildew., brand: Amazon Brand - Inkast Denim Co., weight: , color: L.GREY, height: 1286.0, width: 2342.0, model year: , shape: , style: Sneakers, material: , product_type: SHOES
Product Image: d6/d6058b9c.jpg
Category: SHOES
Country: IN
---
Result 2:
Product Description: men ' s sneakers in black leather with white soles Given Product description: , Warranty Description: 90 days, brand: Amazon Brand - Inkast Denim Co., weight: , color: Grey, height: 1285.0, width: 2342.0, model year: , shape: , style: Sneakers, material: , product_type: SHOES
Product Image: a9/a9e01fe2.jpg
Category: SHOES
Country: IN
---
Result 3:
Product Description: men ' s sneakers - black Given Product description: , Care

#### Generate Natual Language Response using LLM

In [ ]:
!pip install python-dotenv
!pip install openai

In [13]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Check if the API key is loaded correctly
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable.")

In [16]:
from langchain.chat_models import ChatOpenAI

# Load an LLM (GPT-4 for best responses, or use an open-source model)
llm = ChatOpenAI(model_name="gpt-4", temperature=0.7)

def generate_natural_language_response(query, products):
    """Generate a response based on retrieved products using LLM. Include details like the product category and country where it is made."""
    prompt = f"""
    A customer is looking for a product based on this query: "{query}"
    Here are the recommended products:
    {products}
    
    Generate a natural language response listing the products in a friendly tone. Include details like the product category and country where it is made.
    """

    response = llm.predict(prompt)
    return response

# Generate response
response_text = generate_natural_language_response(query, recommended_products)
print(response_text)

Sure, here are some men's sneakers in black that you might like:

1. The first is a pair of men's sneakers in grey and black from the Amazon Brand - Inkast Denim Co. This pair is made in India and comes with a handy tip to use a shoe-horn to avoid damaging the back of your shoes. You can view the image [here](d6/d6058b9c.jpg).

2. The second is another pair from Amazon Brand - Inkast Denim Co. This time, it's a pair of black leather sneakers with white soles. Although the color is listed as grey, the product description mentions black. This pair also comes with a 90-day warranty and is made in India. You can see what they look like [here](a9/a9e01fe2.jpg).

3. The third is a pair of black sneakers from Amazon Brand - Symbol. Made in India, these shoes come with a care instruction to allow your shoes to air and de-odorize at a regular basis. The material is listed as Polyester. Check them out [here](62/62e5bf81.jpg).

4. The fourth and fifth are the same pair of black and white sneakers